## Testing the LESS Denoising Code 

In [1]:
%matplotlib widget

import numpy as np
import h5py
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import os
import cv2
import sys
import pywt

from pathlib import Path
from IPython.display import display
import torch
less_path = os.path.abspath(os.path.join("..", "LESS_ver_alpha"))
if less_path not in sys.path:
    sys.path.insert(0, less_path)
from less import denoise
import seaborn as sns

In [ ]:
def movie_path(mouse, date, file):
    return f"N:/GEVI_Wave/Analysis/Visual/{mouse}/20{date}/{file}/cG_unmixed_dFF.h5"

def mask_movie(movie, raw_mask, binning):
    mask = cv2.resize(raw_mask, (0, 0), fx=1/binning, fy=1/binning, interpolation=cv2.INTER_LINEAR)
    movie_size = movie.shape[1:3]
    mask = mask[:movie_size[0], :movie_size[1]].astype(bool)
    return movie * mask

def run_less_denoising(movie_np, **kwargs):
    assert movie_np.ndim == 3
    movie_tensor = torch.from_numpy(movie_np).float()
    denoised_tensor = denoise(data=movie_tensor, cuda=False, **kwargs)
    return denoised_tensor.numpy()

def denoise_full_movie(mouse, date, file, segment_length=2000, denoise_kwargs={}):
    input_path = f"N:/GEVI_Wave/Analysis/Visual/{mouse}/20{date}/{file}/cG_unmixed_dFF.h5"
    output_path = Path(input_path).with_name("cG_unmixed_dFF_denoised.h5")

    with h5py.File(input_path, 'r') as f_in:
        mov_dset = f_in['mov']
        specs_grp_in = f_in['specs']
        fps = specs_grp_in['fps'][()][0][0][0]
        binning = specs_grp_in['binning'][()][0][0][0]
        raw_mask = specs_grp_in['extra_specs']['mask'][()][0]
        specs_attrs = dict(specs_grp_in.attrs.items())
        extra_specs = {
            k: specs_grp_in["extra_specs"][k][()] for k in specs_grp_in["extra_specs"]
        }

        T, H, W = mov_dset.shape
        with h5py.File(output_path, 'w') as f_out:
            dset_out = f_out.create_dataset('mov', shape=(T, H, W), dtype='float32', compression="gzip")
            specs_grp_out = f_out.create_group("specs")
            for k, v in specs_attrs.items():
                specs_grp_out.attrs[k] = v
            extra_grp_out = specs_grp_out.create_group("extra_specs")
            for k, v in extra_specs.items():
                extra_grp_out.create_dataset(k, data=v)

            for start in range(0, T, segment_length):
                end = min(start + segment_length, T)
                print(f"Denoising frames {start} to {end}")
                segment = mov_dset[start:end]
                segment = np.nan_to_num(segment)
                segment_masked = mask_movie(segment, raw_mask, binning)

                segment_denoised = run_less_denoising(segment_masked, **denoise_kwargs)
                dset_out[start:end] = segment_denoised

    print(f"Denoised movie saved to:\n{output_path}")
   
    return output_path

In [23]:
def denoise_segment(mouse, date, file, segment_length, denoise_kwargs={}):
    input_path = f"N:/GEVI_Wave/Analysis/Visual/{mouse}/20{date}/{file}/cG_unmixed_dFF.h5"

    with h5py.File(input_path, 'r') as f_in:
        mov_dset = f_in['mov']
        specs_grp_in = f_in['specs']
        fps = specs_grp_in['fps'][()][0][0][0]
        binning = specs_grp_in['binning'][()][0][0][0]
        raw_mask = specs_grp_in['extra_specs']['mask'][()][0]

        T, H, W = mov_dset.shape

        segment = mov_dset[:segment_length]
        segment = np.nan_to_num(segment)
        segment_masked = mask_movie(segment, raw_mask, binning)

        segment_denoised = run_less_denoising(segment_masked, **denoise_kwargs)

In [25]:
import time

start_time = time.time()

denoise_segment(
    mouse="cmm002mjr",
    date=231213,
    file="meas03",
    segment_length= 1000,
    denoise_kwargs={
        "patch_size": 16,
        "verbose": False
    })

end_time = time.time()
elapsed = end_time - start_time

print(f"1000 frames: Elapsed time: {elapsed:.2f} seconds")  

start_time = time.time()

denoise_segment(
    mouse="cmm002mjr",
    date=231213,
    file="meas03",
    segment_length= 10000,
    denoise_kwargs={
        "patch_size": 16,
        "verbose": False
    })

end_time = time.time()
elapsed = end_time - start_time

print(f"10000 frames: Elapsed time: {elapsed:.2f} seconds")  

KeyboardInterrupt: 

In [ ]:
import time

start_time = time.time()

denoise_segment(
    mouse="cmm002mjr",
    date=231213,
    file="meas03",
    segment_length= 1000,
    denoise_kwargs={
        "patch_size": 16,
        "verbose": False
    })

end_time = time.time()
elapsed = end_time - start_time

print(f"1000 frames: Elapsed time: {elapsed:.2f} seconds")  

start_time = time.time()

denoise_segment(
    mouse="cmm002mjr",
    date=231213,
    file="meas03",
    segment_length= 10000,
    denoise_kwargs={
        "patch_size": 16,
        "verbose": False
    })

end_time = time.time()
elapsed = end_time - start_time

print(f"10000 frames: Elapsed time: {elapsed:.2f} seconds")  